In [29]:
import json
from tqdm.notebook import tqdm

datas = ''
with open('./bdd100k_labels_images_val.json', 'r', encoding='utf-8') as op:
    datas = json.load(op)
    print(len(datas))

10000


In [32]:
import json
from tqdm.notebook import tqdm

datas = ''
with open('./bdd100k_labels_images_train.json', 'r', encoding='utf-8') as op:
    datas = json.load(op)
    print(len(datas))

69863


In [33]:
datas[0]

{'name': '0000f77c-6257be58.jpg',
 'attributes': {'weather': 'clear',
  'scene': 'city street',
  'timeofday': 'daytime'},
 'timestamp': 10000,
 'labels': [{'category': 'traffic light',
   'attributes': {'occluded': False,
    'truncated': False,
    'trafficLightColor': 'green'},
   'manualShape': True,
   'manualAttributes': True,
   'box2d': {'x1': 1125.902264,
    'y1': 133.184488,
    'x2': 1156.978645,
    'y2': 210.875445},
   'id': 0},
  {'category': 'traffic light',
   'attributes': {'occluded': False,
    'truncated': False,
    'trafficLightColor': 'green'},
   'manualShape': True,
   'manualAttributes': True,
   'box2d': {'x1': 1156.978645,
    'y1': 136.637417,
    'x2': 1191.50796,
    'y2': 210.875443},
   'id': 1},
  {'category': 'traffic sign',
   'attributes': {'occluded': False,
    'truncated': False,
    'trafficLightColor': 'none'},
   'manualShape': True,
   'manualAttributes': True,
   'box2d': {'x1': 1101.731743,
    'y1': 211.122087,
    'x2': 1170.79037,
    

In [34]:
crosswalks = [x for x in datas if any([y['category'] == 'car' for y in x['labels']])]
len(crosswalks)

69072

In [35]:
crosswalks = [x for x in datas if any([y['category'] == 'lane' and y['attributes']['laneType'] == 'crosswalk' for y in x['labels']])]
len(crosswalks)

25624

In [36]:
crosswalks[0]

{'name': '00067cfb-caba8a02.jpg',
 'attributes': {'weather': 'clear',
  'scene': 'highway',
  'timeofday': 'daytime'},
 'timestamp': 10000,
 'labels': [{'category': 'traffic light',
   'attributes': {'occluded': False,
    'truncated': False,
    'trafficLightColor': 'red'},
   'manualShape': True,
   'manualAttributes': True,
   'box2d': {'x1': 610.605671,
    'y1': 298.64355,
    'x2': 619.570103,
    'y2': 329.378736},
   'id': 170},
  {'category': 'traffic light',
   'attributes': {'occluded': False,
    'truncated': False,
    'trafficLightColor': 'red'},
   'manualShape': True,
   'manualAttributes': True,
   'box2d': {'x1': 665.67288,
    'y1': 331.94,
    'x2': 674.637309,
    'y2': 356.272023},
   'id': 171},
  {'category': 'traffic light',
   'attributes': {'occluded': False,
    'truncated': False,
    'trafficLightColor': 'red'},
   'manualShape': True,
   'manualAttributes': True,
   'box2d': {'x1': 121.403968,
    'y1': 334.501266,
    'x2': 132.929663,
    'y2': 357.5526

In [37]:
# get crosswalk
# poly2d -> box2d
# [center x, center y, w, h] (ratio)
# label file name and image name is same(0.jpg, 0.txt)
# store image path(generate label.py)


In [39]:
imageW = 1280.0
imageH = 720.0
for crosswalk in tqdm(crosswalks):
    labelStrings = ''
    f = open("./data/train/" + crosswalk['name'][:-4] + ".txt","w+")
    for label in crosswalk['labels']:
        if label['category'] != 'lane':
            continue
        if label['attributes']['laneType'] != 'crosswalk':
            continue
        minX = imageW
        minY = imageH
        maxX = 0
        maxY = 0
        for poly in label['poly2d']:
            for v in poly['vertices']:
                if minX > v[0]:
                    minX = v[0]
                if minY > v[1]:
                    minY = v[1]
                if maxX < v[0]:
                    maxX = v[0]
                if maxY < v[1]:
                    maxY = v[1]
        meanX = (minX + maxX) / 2
        meanY = (minY + maxY) / 2
        boxW = maxX - minX
        boxH = maxY - minY
        
        f.write(repr(7) + " " + repr(meanX/imageW) + " " + repr(meanY/imageH) + " " + repr(boxW/imageW) + " " + repr(boxH/imageH) + '\n')